To Do:
Clean up result and separate into result, score, and opponent score
Convert Game type to words

In [8]:
import os
import pandas as pd
import sqlalchemy
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [9]:
data = pd.read_sql("SELECT * FROM sports_data.game_stats_2025", conn)

In [10]:
data.columns = [data.columns[i].lower() for i in range(len(data.columns))]

In [11]:
data['Win_Loss'] = data['result'].apply(lambda x: x[:1])

In [12]:
data['Win_Loss'].value_counts()

Win_Loss
L    97223
W    97223
Name: count, dtype: int64

In [13]:
def determine_scores(row):
    if row['Win_Loss'] == 'W':
        row['score'] = row['result'][3:].split('-')[0]
        row['opponent_score'] = row['result'][3:].split('-')[1]
    else:
        row['score'] = row['result'][3:].split('-')[1]
        row['opponent_score'] = row['result'][3:].split('-')[0]
    return row

In [14]:
data = data.apply(determine_scores, axis=1)

In [15]:
data['opponent_score'] = data['result'].apply(lambda x: x[3:].split('-')[1])

In [16]:
data[['result', 'Win_Loss', 'score', 'opponent_score']]

,result,Win_Loss,score,opponent_score
0,"L, 88-57",L,57,57
1,"W, 88-57",W,88,57
2,"L, 83-60",L,60,60
3,"W, 83-60",W,83,60
4,"L, 88-58",L,58,58
...,...,...,...,...
194441,"W, 76-67",W,76,67
194442,"W, 79-64",W,79,64
194443,"L, 79-64",L,64,64
194444,"L, 74-69",L,69,69


In [17]:
for i, row in data.iterrows():
    score = int(row['score'])
    opp_score = int(row['opponent_score'])
    if row['Win_Loss'] == 'W' and opp_score > score:
        print(f"Error at index {i}")
    elif row['Win_Loss'] == 'L' and score > opp_score:
        print(f"Error at index {i}")

In [18]:
# 0 is Non-Conference, 1 is conference, 2 is conference tourney, 3 is NCAA tourney (generally? might be exceptions where 3 is other post season tourney)
def convert_game_type(row):
    if row['game_type'] == 0:
        return 'Non-Conference'
    elif row['game_type'] == 1:
        return 'Conference'
    elif row['game_type'] == 2:
        return 'Conference Tourney'
    elif row['game_type'] == 3:
        return 'Post-Season Tourney'
    else:
        return 'Other'

In [19]:
data['game_type'] = data.apply(convert_game_type, axis=1)

In [20]:
data['game_type'].value_counts()

game_type
Conference             108936
Non-Conference          71318
Conference Tourney      10002
Post-Season Tourney      4190
Name: count, dtype: int64

In [21]:
data.shape[0]

194446

In [22]:
conn.commit()

In [23]:
data.to_sql('game_stats_v3_2025', sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")), if_exists='replace', index=False, schema='sports_data')

446